## Step 3:

In [1]:
import numpy as np
import pandas as pd

In [2]:
# No Need to download data again
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
# print(t)
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
tickers=[]
for stock_group in t:
    for stock_name in stock_group:
        tickers.append(stock_name)

In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\91753\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import os

# Example ticker list and corresponding numeric IDs
ticker_id_map = {t: i + 1 for i, t in enumerate(tickers)}  

# Directory where your CSVs are located
data_dir = ''

# Initialize a list to collect per-ticker-feature DataFrames
stacked_frames = []

# Loop through each ticker file
for t in tickers:
    file_path = os.path.join(data_dir, f"{t}.csv")
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')

    for feature in df.columns:
        temp = df[[feature]].copy()
        temp.columns = pd.MultiIndex.from_tuples([(feature, ticker_id_map[t])])
        stacked_frames.append(temp)

# Concatenate all into one DataFrame
features = pd.concat(stacked_frames, axis=1)

# Sort column levels (feature, stock ID)
features = features.sort_index(axis=1, level=[0, 1])

features = features.drop(columns='Adj Close', level=0)
features.columns = pd.MultiIndex.from_tuples(features.columns)

# Optional: Save to a compact parquet file
features.to_parquet("myfeatures.parquet", engine="pyarrow")


In [5]:
features

Close                                                  \
                   1           2           3           4           5     
Date                                                                     
2020-01-02   82.885262   72.620811   70.934250         NaN   78.908440   
2020-01-03   81.554443   71.914818   70.260933         NaN   77.946472   
2020-01-06   81.795563   72.487854   70.815422         NaN   78.354843   
2020-01-07   82.046280   72.146950   70.411438         NaN   77.919220   
2020-01-08   82.856331   73.307518   70.910477         NaN   78.236870   
...                ...         ...         ...         ...         ...   
2023-12-22  138.080872  192.192551  146.723526  140.800003  105.848404   
2023-12-26  138.318283  191.646576  146.420502  138.720001  106.042580   
2023-12-27  138.328186  191.745804  146.666718  136.550003  106.586235   
2023-12-28  138.278717  192.172714  146.543610  137.000000  107.178436   
2023-12-29  137.779251  191.130325  146.751953  136.139999  106.858063   

                                                                      ...  \
                  6           7           8           9          10   ...   
Date                                                                  ...   
2020-01-02  41.268997  193.477844  334.429993  109.436218  39.462341  ...   
2020-01-03  41.221451  193.155640  331.809998  107.509750  39.385307  ...   
2020-01-06  41.383106  191.894333  333.709991  106.246651  39.077213  ...   
2020-01-07  41.040779  187.751328  333.390015  108.663826  38.606503  ...   
2020-01-08  40.631893  188.119568  337.869995  109.645226  38.178589  ...   
...               ...         ...         ...         ...        ...  ...   
2023-12-22  70.214340  345.651978  598.750000  192.972641  67.514664  ...   
2023-12-26  69.900551  344.657288  598.260010  194.705566  68.393326  ...   
2023-12-27  69.929077  345.037628  596.080017  195.175522  68.185463  ...   
2023-12-28  70.423546  342.862915  595.520020  196.046906  68.279945  ...   
2023-12-29  70.623230  342.199799  596.599976  194.402069  68.232704  ...   

            Volume                                                         \
               494      495      496      497       498      499      500   
Date                                                                        
2020-01-02  465800  4973500  3006500  2915400  12456400   869500  1369900   
2020-01-03  411400  3111000  1760500  2422500  17386900   795100  1145500   
2020-01-06  620200  2178600  1357900  2217100  20081900   817300  1454100   
2020-01-07  443200  2426200  1722100  1861100  17387700   662200  1388600   
2020-01-08  486400  2299800  1176200  4828900  15137700  1083200  1521500   
...            ...      ...      ...      ...       ...      ...      ...   
2023-12-22  328600  2253500  1170500  2429300  12921800   829300   991500   
2023-12-26  296900  2542300  1150000  2147500  16835100   440300   627500   
2023-12-27  323300  2109200  1142200  2865900  14558800  1007700  1050400   
2023-12-28  265200  2586100  1233000  2421900  16329300   709100   882500   
2023-12-29  336900  3079100  1144800  2813300  17741400   698900  1196800   

                                      
                501     502      503  
Date                                  
2020-01-02  1083972  387800  1576700  
2020-01-03   992405  305300  1274000  
2020-01-06   972423  322600  2334100  
2020-01-07   934519  236300  1224500  
2020-01-08   936064  689600  1766700  
...             ...     ...      ...  
2023-12-22  1337800  225900  1548400  
2023-12-26  1870700  220000   814600  
2023-12-27  1058600  275700   766400  
2023-12-28   662200  193900   880100  
2023-12-29   849600  251300  1007200  

[1006 rows x 2515 columns]

In [6]:
# Extract stock IDs and dates from the features DataFrame
stocks = features.columns.levels[1]  # 503 stock IDs
dates = features.index              # 1006 dates

# Create DataFrame
binary_matrix = pd.DataFrame(
    data=1,
    index=dates,
    columns=stocks
)

binary_matrix.to_parquet("myuniverse.parquet", engine="pyarrow")



In [7]:
binary_matrix

,1,2,3,4,5,6,7,8,9,10,...,494,495,496,497,498,499,500,501,502,503
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2020-01-03,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2020-01-06,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2020-01-07,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2020-01-08,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2023-12-26,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2023-12-27,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
